In [12]:
!pip3 install xgboost lightgbm catboost
!pip3 install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 258 kB 6.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.covariance import EllipticEnvelope
from snowflake.snowpark import Session
import config
import random
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import config
import random

# Connect to Snowflake
def get_snowflake_session():
    connection_parameters = {
        "account": config.SNOWFLAKE_ACCOUNT,
        "user": config.SNOWFLAKE_USER,
        "role": config.SNOWFLAKE_ROLE,
        "warehouse": config.SNOWFLAKE_WAREHOUSE,
        "database": config.SNOWFLAKE_DATABASE,
        "schema": config.SNOWFLAKE_SCHEMA,
        "authenticator": config.SNOWFLAKE_AUTHENTICATOR
    }
    session = Session.builder.configs(connection_parameters).create()
    print(f"Connected to Snowflake as {config.SNOWFLAKE_USER}")
    return session

session = get_snowflake_session()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://blinkist-useast_1_virginia.snowflakecomputing.com/console/login?login_name=meri-kris.jaama%40go1.com&browser_mode_redirect_port=50960&proof_key=XvD294gbGfa78I8goietmkjialzUv7kbMuD98Otr8AY%3D to authenticate...
Connected to Snowflake as meri-kris.jaama@go1.com


In [3]:
# Load data from Snowflake
query = "SELECT * FROM blinkist_dev.dbt_mjaama.exp_proceeds_input"
df = session.sql(query).to_pandas()
df.columns = df.columns.str.lower()

from country_utils import add_signup_country_group
from data_utils import split_data_by_user_type

df, _, _ = split_data_by_user_type(df)

# Add country groups
df = add_signup_country_group(df)

/Users/meri-kris/Documents/GitHub/blinkist-expected-proceeds/country_utils.py:37: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  us_df = temp_df.loc[(temp_df.report_date >= six_months_ago) & (temp_df.signup_country == "US") & (
/Users/meri-kris/Documents/GitHub/blinkist-expected-proceeds/country_utils.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  row_df = temp_df.loc[(temp_df.report_date >= six_months_ago) & (


In [4]:
# Ensure date column is datetime
df['report_date'] = pd.to_datetime(df['report_date'])

# Define the target for trial conversion
df['trial_conversion'] = (df['eur_proceeds_d8'] > 0).astype(int)
target = 'trial_conversion'

# Define features and target
categorical_features = ['channel_group', 'marketing_network_id', 'target_market',
                        'signup_country_group', 'signup_client_platform', 'plan_tier']
numerical_features = ['eur_marketing_spend', 'impressions', 'clicks', 
                      'is_trial_autorenewal_on_d0', 'started_content', 
                      'n_content_starts', 'finished_content', 'n_content_finishes']
features = categorical_features + numerical_features

In [5]:
# Initial distribution of the target variable
initial_distribution = df['trial_conversion'].value_counts(normalize=True)
print("Initial target distribution:\n", initial_distribution)

# Initial number of rows
initial_row_count = df.shape[0]

# Outlier detection and removal
outlier_detector = EllipticEnvelope(contamination=0.01)
outliers = outlier_detector.fit_predict(df[numerical_features])
df = df[outliers == 1]

# Number of rows after outlier removal
final_row_count = df.shape[0]

# Final distribution of the target variable
final_distribution = df['trial_conversion'].value_counts(normalize=True)
print("Final target distribution:\n", final_distribution)

# Log the number of rows removed
rows_removed = initial_row_count - final_row_count
print(f"Number of rows removed during outlier detection: {rows_removed}")

Initial target distribution:
 0    0.602477
1    0.397523
Name: trial_conversion, dtype: float64


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (33.947994939261214 > 5.724941626988796). You may want to try with a higher value of support_fraction (current value: 0.503).
  warnings.warn(
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (33.499825063469331 > 3.485799777977076). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (32.939019348992169 > -3.687650784476178). You may want to try with a higher value of support_fraction (c

Final target distribution:
 0    0.602962
1    0.397038
Name: trial_conversion, dtype: float64
Number of rows removed during outlier detection: 9000


In [6]:
# Define the number of sets and the window sizes
num_sets = 5
train_window_days = 180
test_window_days = 2
gap_days = 9

# Sort data by date
df = df.sort_values(by='report_date')

# Get the date range for the past three months
max_date = df['report_date'].max() - pd.Timedelta(days=8)
three_months_ago = max_date - pd.Timedelta(days=90)

# Generate random test start dates
random_dates = pd.date_range(start=three_months_ago, end=max_date - pd.Timedelta(days=test_window_days), freq='D')
random_test_dates = random.sample(list(random_dates), num_sets)

# Generate train and test sets
train_test_sets = []

for test_start_date in random_test_dates:
    test_end_date = test_start_date + pd.Timedelta(days=test_window_days)
    train_end_date = test_start_date - pd.Timedelta(days=gap_days)
    train_start_date = train_end_date - pd.Timedelta(days=train_window_days)
    
    train_df = df[(df['report_date'] >= train_start_date) & (df['report_date'] < train_end_date)]
    test_df = df[(df['report_date'] >= test_start_date) & (df['report_date'] < test_end_date)]
    
    train_test_sets.append((train_df, test_df))

In [7]:
# Train and evaluate models on each set
results = []

for idx, (train_df, test_df) in enumerate(train_test_sets):
    
    # Calculate and print the distribution of the actual target variable
    train_distribution = train_df[target].value_counts(normalize=True)
    test_distribution = test_df[target].value_counts(normalize=True)
    print(f"  Train target distribution:\n{train_distribution}")
    print(f"  Test target distribution:\n{test_distribution}")
    
    

  Train target distribution:
0    0.780693
1    0.219307
Name: trial_conversion, dtype: float64
  Test target distribution:
0    0.848073
1    0.151927
Name: trial_conversion, dtype: float64
  Train target distribution:
0    0.720236
1    0.279764
Name: trial_conversion, dtype: float64
  Test target distribution:
0    0.79
1    0.21
Name: trial_conversion, dtype: float64
  Train target distribution:
0    0.721241
1    0.278759
Name: trial_conversion, dtype: float64
  Test target distribution:
0    0.787938
1    0.212062
Name: trial_conversion, dtype: float64
  Train target distribution:
0    0.763785
1    0.236215
Name: trial_conversion, dtype: float64
  Test target distribution:
0    0.82521
1    0.17479
Name: trial_conversion, dtype: float64
  Train target distribution:
0    0.747751
1    0.252249
Name: trial_conversion, dtype: float64
  Test target distribution:
0    0.779343
1    0.220657
Name: trial_conversion, dtype: float64


In [8]:
# Preprocessing pipelines
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Define classification models
models = {
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False),
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(random_state=42, verbose=0)
}



# Parameter grids for hyperparameter tuning
param_grids = {
    'RandomForest': {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10]
    },
    'GradientBoosting': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    },
    'XGBoost': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    },
    'LightGBM': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [-1, 10, 20]
    },
    'CatBoost': {
        'model__iterations': [100, 200, 300],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__depth': [3, 5, 7]
    }
}


In [18]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import time

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Define categorical and numerical features
categorical_features = ['channel_group', 'marketing_network_id', 'target_market',
                        'signup_country_group', 'signup_client_platform', 'plan_tier']
numerical_features = ['eur_marketing_spend', 'impressions', 'clicks', 
                      'is_trial_autorenewal_on_d0', 'started_content', 
                      'n_content_starts', 'finished_content', 'n_content_finishes']

# Preprocessing pipelines
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])


In [19]:
# Train and evaluate models on each set
results = []

for idx, (train_df, test_df) in enumerate(train_test_sets):
    print(f"Processing set {idx+1}/{len(train_test_sets)}")
    
    # Calculate and print the distribution of the actual target variable
    train_distribution = train_df[target].value_counts(normalize=True)
    test_distribution = test_df[target].value_counts(normalize=True)
    print(f"  Train target distribution:\n{train_distribution}")
    print(f"  Test target distribution:\n{test_distribution}")
    
    X_train, y_train = train_df[features], train_df[target]
    X_test, y_test = test_df[features], test_df[target]
    
    # Apply preprocessing to the training and test data
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_preprocessed = preprocessor.transform(X_test)
    
    # Apply SMOTE to balance the training data
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)
    
    for name, model in models.items():
        print(f"  Training model: {name}")
        start_time = time.time()
        pipeline = Pipeline(steps=[('model', model)])
        
        if name in param_grids:
            print(f"    Performing grid search for {name}")
            grid_search = GridSearchCV(pipeline, param_grids[name], cv=3, scoring='f1')
            grid_search.fit(X_train_resampled, y_train_resampled)
            best_model = grid_search.best_estimator_
            print(f"    Best parameters for {name}: {grid_search.best_params_}")
        else:
            best_model = pipeline.fit(X_train_resampled, y_train_resampled)
        
        y_pred = best_model.predict(X_test_preprocessed)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        results.append({'Set': idx+1, 'Model': name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1})
        elapsed_time = time.time() - start_time
        print(f"  Completed {name} in {elapsed_time:.2f} seconds - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Processing set 1/5
  Train target distribution:
0    0.780693
1    0.219307
Name: trial_conversion, dtype: float64
  Test target distribution:
0    0.848073
1    0.151927
Name: trial_conversion, dtype: float64


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


  Training model: XGBoost
    Performing grid search for XGBoost


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [14:41:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [14:41:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [14:41:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [14:41:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

    Best parameters for XGBoost: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 200}
  Completed XGBoost in 38.34 seconds - Accuracy: 0.7142857142857143, Precision: 0.2814814814814815, Recall: 0.5671641791044776, F1 Score: 0.37623762376237624
  Training model: RandomForest
    Performing grid search for RandomForest
    Best parameters for RandomForest: {'model__max_depth': None, 'model__min_samples_split': 2, 'model__n_estimators': 200}
  Completed RandomForest in 1685.12 seconds - Accuracy: 0.800453514739229, Precision: 0.3670886075949367, Recall: 0.43283582089552236, F1 Score: 0.3972602739726027
  Training model: GradientBoosting
    Performing grid search for GradientBoosting
    Best parameters for GradientBoosting: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100}
  Completed GradientBoosting in 1166.75 seconds - Accuracy: 0.7346938775510204, Precision: 0.29838709677419356, Recall: 0.5522388059701493, F1 Score: 0.3874345

...

[LightGBM] [Info] Number of positive: 35792, number of negative: 35791
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006413 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3977
[LightGBM] [Info] Number of data points in the train set: 71583, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500007 -> initscore=0.000028
[LightGBM] [Info] Start training from score 0.000028


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 53687, number of negative: 53687
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009073 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5031
[LightGBM] [Info] Number of data points in the train set: 107374, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
    Best parameters for LightGBM: {'model__learning_rate': 0.1, 'model__max_depth': -1, 'model__n_estimators': 100}
  Completed LightGBM in 82.32 seconds - Accuracy: 0.784037558685446, Precision: 0.5100671140939598, Recall: 0.5390070921985816, F1 Score: 0.5241379310344828
  Training model: CatBoost
    Performing grid search for CatBoost


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


    Best parameters for CatBoost: {'model__depth': 3, 'model__iterations': 300, 'model__learning_rate': 0.1}
  Completed CatBoost in 119.04 seconds - Accuracy: 0.7856025039123631, Precision: 0.5142857142857142, Recall: 0.5106382978723404, F1 Score: 0.5124555160142349


In [ ]:
# Calculate average performance for each model
average_results = {}

for result in results:
    model_name = result['Model']
    if model_name not in average_results:
        average_results[model_name] = {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1 Score': []}
    average_results[model_name]['Accuracy'].append(result['Accuracy'])
    average_results[model_name]['Precision'].append(result['Precision'])
    average_results[model_name]['Recall'].append(result['Recall'])
    average_results[model_name]['F1 Score'].append(result['F1 Score'])

for model_name, metrics in average_results.items():
    avg_accuracy = np.mean(metrics['Accuracy'])
    avg_precision = np.mean(metrics['Precision'])
    avg_recall = np.mean(metrics['Recall'])
    avg_f1 = np.mean(metrics['F1 Score'])
    average_results[model_name] = {
        'Avg Accuracy': avg_accuracy,
        'Avg Precision': avg_precision,
        'Avg Recall': avg_recall,
        'Avg F1 Score': avg_f1
    }
    print(f"{model_name} - Avg Accuracy: {avg_accuracy}, Avg Precision: {avg_precision}, Avg Recall: {avg_recall}, Avg F1 Score: {avg_f1}")


In [20]:
# Select the best model based on average MSE
best_model_name = min(average_results, key=lambda x: average_results[x]['Avg MSE'])
best_model_metrics = average_results[best_model_name]

print(f"Best model: {best_model_name} with Avg MSE: {best_model_metrics['Avg MSE']} and Avg MAE: {best_model_metrics['Avg MAE']}")

NameError: name 'average_results' is not defined

In [ ]:
# Close Snowflake session
session.close()
print("Snowflake session closed")